# Mini-projet – Débruitage de maillage 3D

IMI – Optimisation et problèmes inverses  
S8 – 2024/2025  

## Objectif

On cherche à débruiter un maillage triangulé en résolvant le problème :


xhat = argmin_(x in {R}^{N**3}) 
[||x - z||^2 + lambda R(x)]

où :
- \( z \) est le maillage bruité
- \( R(x) \) est une régularisation sur le graphe du maillage
- \( lambda \) est un paramètre de régularisation


# Imports

In [8]:
# ===============================
# Bibliothèques Maths
# ===============================
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as spla

# ===============================
# Processing 3D
# ===============================
import igl  # libigl python binding

# ===============================
# Visualisation
# ===============================
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# ===============================
# IMICPE
# ===============================
import imicpe
import imicpe.optim as optim

print("imicpe version:", imicpe.__version__)


imicpe version: 1.0.15


# Lecture du maillage

In [9]:
# Chemin vers le maillage
mesh_path = "sphere.off"

# Lecture
V, F = igl.read_triangle_mesh(mesh_path)

print("Nombre de sommets :", V.shape[0])
print("Nombre de faces :", F.shape[0])

Nombre de sommets : 162
Nombre de faces : 320


# Visualisation du maillage

In [10]:
# def plot_mesh(V, F, title="Mesh"):
#     x, y, z = V[:, 0], V[:, 1], V[:, 2]
#     i, j, k = F[:, 0], F[:, 1], F[:, 2]

#     fig = go.Figure(data=[go.Mesh3d(
#         x=x, y=y, z=z,
#         i=i, j=j, k=k,
#         color='cyan',
#         opacity=0.5
#     )])

#     fig.update_layout(title=title, scene=dict(aspectmode='data'))
#     fig.show()


def plot_mesh(V, F, title="Mesh"):
    """
    V : matrice (n,3) des sommets
    F : matrice (m,3) des faces (indices des sommets)
    """

    x, y, z = V[:, 0], V[:, 1], V[:, 2]
    i, j, k = F[:, 0], F[:, 1], F[:, 2]

    fig = go.Figure()

    # --- Surface avec dégradé ---
    fig.add_trace(go.Mesh3d(
        x=x,
        y=y,
        z=z,
        i=i,
        j=j,
        k=k,
        intensity=z,                 # dégradé basé sur la hauteur
        colorscale='Viridis',        # palette de couleurs
        opacity=0.9,
        showscale=True
    ))

    # --- Construction des arêtes ---
    edges_x = []
    edges_y = []
    edges_z = []

    for tri in F:
        for a, b in [(0, 1), (1, 2), (2, 0)]:
            edges_x += [V[tri[a], 0], V[tri[b], 0], None]
            edges_y += [V[tri[a], 1], V[tri[b], 1], None]
            edges_z += [V[tri[a], 2], V[tri[b], 2], None]

    fig.add_trace(go.Scatter3d(
        x=edges_x,
        y=edges_y,
        z=edges_z,
        mode='lines',
        line=dict(color='black', width=2),
        showlegend=False
    ))

    # --- Mise en page ---
    fig.update_layout(
        title=title,
        scene=dict(aspectmode='data')
    )

    fig.show()

plot_mesh(V, F, "Maillage original")

# Ajout de bruit Gaussien

In [11]:
VB = np.zeros((len(V), len(V[0])))
for i in range(len(V)) :
    n = np.random.normal(0, 0.1, len(V[i]))
    xbar = np.array(V[i])
    z = xbar + n
    VB[i] = z

plot_mesh(VB, F, "Maillage bruité")

# Fonction de coût

In [12]:
# attache aux données
def f(x):
    return np.sum((x - z)**2)

# régularisation
def RL(x):
    return np.sum(optim.L(x)**2)

# fonction de coût globale
def E(x,lam):
    return f(x) + lam * RL(x)

# # opérateur proximal de la norme l1
# def prox_l1(x, gamma, lam):
#     return np.sign(x) * np.maximum(np.abs(x) - gamma*lam, 0.0)

# def prox_l2(x, gamma):
#     return x / (1 + 2*gamma)

# def prox_f(x, gamma, lam):
#     """
#     Proximal de f(x) = x^2 + lam |x|
#     """
#     y = prox_l2(x, gamma)
#     return prox_l1(y, gamma, lam)


# Algorithme

In [13]:
lam = 0.1
tk = 0.01
Niter = 1000

xn = np.zeros(VB.shape)
En = []

for k in range(Niter):
    grad = 2*(xn - VB) + 2*lam*optim.Lt(optim.L(xn))
    xn = xn - tk * grad
    
    energy = np.sum((xn - VB)**2) + lam*np.sum(optim.L(xn)**2)
    En.append(energy)

VF = xn

# Affichage des résultats

In [14]:
# Fonction de coût en fonction des itérations
plt_cost = pd.DataFrame({'x':np.arange(len(En)), 'y':En, 'legend':'cost'})

fig = px.line(plt_cost,
              x='x', 
              y='y', 
              log_x=True, log_y=True,
              labels={'x':'itérations (logscale)','y':'E(x^k) (logscale)'},
              title='Évolution de la fonction de coût en fonction des itérations',
              width=800, height=350)
fig.show()

# Approximations finales
plot_mesh(VF, F, "Maillage débruité")